In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 設置數據生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    validation_split=0.2    
)

# 訓練數據生成器
train_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\Garbage',  # 替換為您的資料路徑
    target_size=(224, 224),  
    batch_size=16,
    class_mode='categorical',
    subset='training'  # 指定為訓練集
)

# 驗證數據生成器
validation_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\Garbage',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation'  # 指定為驗證集
)

Found 6465 images belonging to 4 classes.
Found 1614 images belonging to 4 classes.


In [4]:
# 測試層數
unfreeze_layers = [1 ,2, 3, 4, 5, 6, 7, 8, 9]

# 逐步測試不同的解凍層數
for unfreeze in unfreeze_layers:
    print(f"\n==== 測試解凍層數: {unfreeze} ====")
    
    # 加載預訓練模型
    base_model = MobileNetV2(
        alpha=0.35, 
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )

    # 添加自定義分類層
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(4, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=x)

    # 設定解凍層
    for layer in base_model.layers:
        layer.trainable = False
        
    for layer in base_model.layers[:unfreeze]:  
        layer.trainable = True

    # 編譯模型
    model.compile(
        optimizer='Adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )

    early_stopping = EarlyStopping(
        monitor='val_accuracy', # 監控驗證準確率
        patience=5,             # 連續 5 個 epoch 沒有提升就停止
        mode='max',             # 監控的指標是越大越好
        restore_best_weights=True # 停止時恢復最佳權重
    )
    # 訓練模型
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=50,
        steps_per_epoch=train_generator.samples // 16,
        validation_steps=validation_generator.samples // 16,
        callbacks=[early_stopping]
    )


==== 測試解凍層數: 1 ====
Epoch 1/50
320/404 [======================>.......] - ETA: 15s - loss: 0.4202 - accuracy: 0.8317

KeyboardInterrupt: 

In [ ]:
# 測試層數
big_unfreeze_layers = [10 ,20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150]

# 逐步測試不同的解凍層數
for unfreeze in big_unfreeze_layers:
    print(f"\n==== 測試解凍層數: {unfreeze} ====")
    
    # 加載預訓練模型
    base_model = MobileNetV2(
        alpha=0.35, 
        weights='imagenet', 
        include_top=False, 
        input_shape=(224, 224, 3)
    )

    # 添加自定義分類層
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(4, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=x)

    # 設定解凍層
    for layer in base_model.layers:
        layer.trainable = False
        
    for layer in base_model.layers[:unfreeze]:  
        layer.trainable = True

    # 編譯模型
    model.compile(
        optimizer='Adam', 
        loss='categorical_crossentropy', 
        metrics=['accuracy']
    )

    early_stopping = EarlyStopping(
        monitor='val_accuracy', # 監控驗證準確率
        patience=5,             # 連續 5 個 epoch 沒有提升就停止
        mode='max',             # 監控的指標是越大越好
        restore_best_weights=True # 停止時恢復最佳權重
    )
    
    # 訓練模型
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=50,
        steps_per_epoch=train_generator.samples // 16,
        validation_steps=validation_generator.samples // 16,
        callbacks=[early_stopping]
    )


==== 測試解凍層數: 10 ====
Epoch 1/10
404/404 [==============================] - 66s 160ms/step - loss: 0.3824 - accuracy: 0.8421 - val_loss: 1.3407 - val_accuracy: 0.5163
Epoch 2/10
404/404 [==============================] - 65s 161ms/step - loss: 0.2574 - accuracy: 0.8997 - val_loss: 0.7385 - val_accuracy: 0.7044
Epoch 3/10
404/404 [==============================] - 65s 160ms/step - loss: 0.2108 - accuracy: 0.9174 - val_loss: 0.7900 - val_accuracy: 0.7150
Epoch 4/10
404/404 [==============================] - 65s 160ms/step - loss: 0.1857 - accuracy: 0.9307 - val_loss: 0.5218 - val_accuracy: 0.7950
Epoch 5/10
404/404 [==============================] - 65s 160ms/step - loss: 0.1759 - accuracy: 0.9330 - val_loss: 0.4474 - val_accuracy: 0.8350
Epoch 6/10
404/404 [==============================] - 65s 160ms/step - loss: 0.1540 - accuracy: 0.9406 - val_loss: 0.5996 - val_accuracy: 0.7738
Epoch 7/10
404/404 [==============================] - 64s 159ms/step - loss: 0.1455 - accuracy: 0.9451 - val

In [4]:
# 測試層數
dropout_rates = [0.1, 0.2, 0.3, 0.4, 0.5]
dense_units = [16, 32, 64, 128, 256]

for dropout_rate in dropout_rates:
    for dense_unit in dense_units:
        print(f"\n== 測試 Dropout: {dropout_rate}, Dense: {dense_unit} ==")

        # 加載 MobileNetV2 預訓練模型
        base_model = MobileNetV2(
            alpha=0.35,
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )

        # 添加自定義分類層
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dropout(dropout_rate)(x)
        x = Dense(dense_unit, activation='relu')(x)
        x = Dense(4, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=x)

        # 凍結所有 base_model 的層
        for layer in base_model.layers:
            layer.trainable = False
        

        # 編譯模型
        model.compile(
            optimizer='Adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # 訓練模型
        history = model.fit(
            train_generator,
            validation_data=validation_generator,
            epochs=10,  # 測試可先設定較少 epochs
            steps_per_epoch=train_generator.samples // 16,
            validation_steps=validation_generator.samples // 16,
            verbose=1
        )


== 測試 Dropout: 0.1, Dense: 16 ==
Epoch 1/10
404/404 [==============================] - 61s 147ms/step - loss: 0.3964 - accuracy: 0.8412 - val_loss: 0.3798 - val_accuracy: 0.8381
Epoch 2/10
404/404 [==============================] - 58s 144ms/step - loss: 0.2250 - accuracy: 0.9161 - val_loss: 0.3768 - val_accuracy: 0.8594
Epoch 3/10
404/404 [==============================] - 61s 150ms/step - loss: 0.1971 - accuracy: 0.9260 - val_loss: 0.4263 - val_accuracy: 0.8537
Epoch 4/10
404/404 [==============================] - 59s 146ms/step - loss: 0.1836 - accuracy: 0.9277 - val_loss: 0.3494 - val_accuracy: 0.8700
Epoch 5/10
404/404 [==============================] - 59s 145ms/step - loss: 0.1609 - accuracy: 0.9370 - val_loss: 0.3456 - val_accuracy: 0.8662
Epoch 6/10
404/404 [==============================] - 58s 145ms/step - loss: 0.1480 - accuracy: 0.9436 - val_loss: 0.3738 - val_accuracy: 0.8725
Epoch 7/10
404/404 [==============================] - 58s 143ms/step - loss: 0.1424 - accuracy: 

KeyboardInterrupt: 